### **[RUBRIC FEEDBACK]**

**[Algemeen]:** Over het algemeen een hele nette uitleg en data preparatie, goed gedaan! De visualisaties zijn ook netjes, maar kunnen nog iets aangevuld/uitgelegd worden.

**[Visualisaties] (__Op Niveau__):** Je hebt redelijk veel (goede) visualisaties, deze zien er al mooi uit. Echter mis ik nog wel wat afwerking van de plots door de redenering erachter uit te leggen en de assen netjes op te maken.

**[Data preparatie] (__Op Niveau__):** Je mist data-exploratie en bijna alle preparatie, je legt wel goed uit wat je doet en waarom. Ook is de code netjes en overzichtelijk.

**[Gebruikte technieken] (__Op Niveau__):** Je gebruikt zeker voldoende technieken en tools om je data te analyseren en visualiseren.

---
#### **[FEEDBACK]**

Hele nette introductie van het onderwerp, je toont duidelijk wat je gaat doen, en dit is relevant voor de lezer.
</b>

### Introductie

Met behulp van visualisaties ga ik het volgende proberen aan te tonen over de airbnb listings in Amsterdam:

- Of de locatie/wijk waarin een airbnb zich bevindt invloed heeft op de prijs.

- Of er een correlatie is tussen de prijs en het aantal reviews van een airbnb.

Deze informatie kan handig zijn voor iemand die overweegt een airbnb te huren in Amsterdam, en op zoek is naar een redelijke prijs.

### Aanpak

Eerst ga ik kijken naar alle kolommen die mogelijk relevant zijn voor het onderzoek. Ik ga kijken of deze kolommen waarden missen en/of uitschieters hebben. Als een relevante kolom veel missende waarden heeft zal ik hier een oplossing voor bedenken en toepassen. Als een relevante kolom (extreme) uitschieters bevat zal ik deze uit de dataset droppen.

### Relevante kolommen

Eerst laad ik de dataset in, en kijk ik naar de kolomnamen en een paar rijen data. Ik kies de kolommen die mogelijk bruikbaar zijn voor het onderzoek:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
pd.set_option('display.max_columns', None)

In [ ]:
airbnb = pd.read_csv("../databronnen/listings.csv.gz")  # Path aangepast door Jesse voor het inladen van data lokaal

selectie_kolommen = ['id', 'listing_url', 'name',
       'description', 'host_id',
       'host_name', 'host_since', 
       'host_is_superhost', 'host_listings_count',
       'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed','latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights',  'has_availability',
       'availability_30', 'number_of_reviews',
       'number_of_reviews_ltm', 
       'review_scores_rating', 'license']

airbnb = airbnb[selectie_kolommen]

airbnb.head()

#### **[FEEDBACK]**

Nette en code en goed opgesplitst. Om dit helemaal af te maken kan je in de markdown over de gekozen kolommen nog wat redenering toevoegen; 'Waarom heb ik deze kolom gekozen?'.

---

De kolommen die ik voor het onderzoek gekozen heb zijn de volgende:  
- neighbourhood (de plek waar de airbnb zich bevindt)
- neighbourhood_cleansed (de wijk waarin de airbnb zich bevindt, preciezer dan neighbourhood)
- price (prijs per nacht)
- number_of_reviews (aantal reviews)

### Missende waarden

Ik ga nu onderzoeken of er missende waarden zijn in deze kolommen.





In [ ]:
airbnb.info()

#### **[FEEDBACK]**

Hele uitgebreide en goede uitleg over de missende waarden, goed gedaan! 

Ik adviseer je wel om niet vroeg conclusies te trekken over de kolommen aangezien een aantrekkelijke kolom mogelijk 50% uitschieters kan hebben, waardoor een andere kolom toch nuttiger kan blijken, maar dit is iets wat je niet overmatig doet, dus goed bezig!

---

-neighbourhood  
De kolom mist 4910 waarden, wat behoorlijk veel is. Als ik deze kolom wil gebruiken is het geen goed idee om de missende waarden aan te vullen met bijvoorbeeld de modus, omdat dit de onderzoeksresultaten aanzienlijk zal vertekenen. Het zou beter zijn om de percentages van elke categorie te berekenen en de missende waarden met deze verdeling op te vullen.

-neighbourhood_cleansed  
Deze kolom mist geen waarden, en kan direct gebruikt worden.

-price  
Deze kolom mist 3847 waarden. Een rede hiervoor zou kunnen zijn dat de prijs variabel is, afhankelijk van bijvoorbeeld de vakantieperiode's. Ik kies ervoor om alle rijen met een missende waarde weg te gooien. Dit is een aanzienlijk deel van de data, maar ik denk dat 6321 rijen meer dan genoeg is om goede conclusies te kunnen trekken, en op deze manier hoef ik de missende waarden niet aan te vullen en blijft de data zo betrouwbaar mogelijk.

-number_of_reviews  
Deze kolom mist ook geen waarden, en kan direct gebruikt worden

Ik heb besloten om de kolom 'neighbourhood' niet te gebruiken. 'neighbourhood_cleansed' is een veel betere kolom, omdat deze geen missende waarden heeft en de locatie preciezer aangeeft dan 'neighbourhood'.

### Outliers

Voor de numerieke kolommen 'price' en 'number_of_reviews' is het belangrijk om te kijken of er uitschieters zijn. Dit doe ik met behulp van boxplots.

De price kolom bevat nu nog strings met een dollar-teken voor de prijs. Ik ga eerst het dollar-teken verwijderen en de waarden omzetten naar floats, zodat er mee gerekend kan worden.

#### **[FEEDBACK]**

Netjes dat je de kolom `price` hier niet overschrijft.

In [ ]:
airbnb['price_numeric'] = airbnb['price'].str.replace('$','').str.replace(',','').astype(float)

In [ ]:
airbnb.boxplot(["price_numeric"])
plt.title("Boxplot Airbnb prijzen")
plt.show()

Zoals te zien is, zijn er een paar extreme uitschieters die de data vertekenen. Het zou goed kunnen dat deze hoge prijzen verkeerd ingevuld zijn. Ik ga deze uitschieters uit data verwijderen door te filteren op een prijs lager dan 8000:

#### **[FEEDBACK]**

Wat je hier doet is niet per se fout, maar ik zou erg oplettend zijn met data wegsnijden uit je basisdataset. Als je bijvoorbeeld later de uitschieters wilt bekijken kan dit niet meer. Misschien is het versandiger om een kopie te maken van het DataFrame en hieruit de uitschieters uit te snijden.

**[Feedback TJM]** Kijk of je hier een onderbouwde manier kan vinden om uitschieters te verwijderen. Wat we al hebben behandeld is gebuik maken van de IQR (dus bovengrens = Q3 + 1.5 * IQR), maar er zijn (veel) meer methoden om dit goed te doen. 

In [ ]:
airbnb = airbnb[airbnb["price_numeric"] < 8000]

In [ ]:
airbnb.boxplot(["price_numeric"])
plt.title("Boxplot Airbnb prijzen")
plt.show()

Er zijn nog best veel datapunten die (ver) boven de box uitkomen, maar deze prijzen zijn op zich te geloven omdat Amsterdam een populaire toeristische stad is, en er veel dure panden staan.

Voor de 'number_of_reviews' kolom maak ik ook een boxplot:

In [ ]:
airbnb.boxplot(["number_of_reviews"])
plt.title("Boxplot number_of_reviews")
plt.show()

Ook hier zijn een paar uitschieters te zien. Een mogelijke verklaring zou kunnen zijn dat de host gesjoemeld heeft, en zelf reviews heeft achtergelaten op zijn/haar airbnb. Dit is echter niet zeker. Ik kies ervoor om de grootste uitschieters te verwijderen door te filteren op een aantal reviews lager dan 2000:

In [ ]:
airbnb = airbnb[airbnb["number_of_reviews"] < 2000]

In [ ]:
airbnb.boxplot(["number_of_reviews"])
plt.title("Boxplot number_of_reviews")
plt.show()

Ook voor deze kolom zijn er nog best veel datapunten die (ver) buiten de box liggen. Ik vind dat deze datapunten nog wel betrouwbaar genoeg zijn om erin te laten. Airbnb bestaat al 17 jaar, wat het realistisch maakt dat sommige airbnb's meer dan 1000 reviews hebben.

### Visualisatie 1

Met deze barplot probeer ik te laten zien of er verschil in gemiddelde prijs zit tussen wijken:

#### **[FEEDBACK]**

Technisch gezien is deze grafiek correct, maar er zijn een paar punten die verbeterd kunnen worden:
- De labels op de x-as zijn moeilijk leesbaar door de hoek en lengte, je kan mogelijk de assen omdraaien en/of onnodige informatie weghalen zoals `Amsterdam` en `, Netherlands`.
- Er is geen eenheid (valutasymbool) bij de y-as, wat handig kan zijn om toe te voegen.
- Sommige waarden worden uit proportie getrokken aangezien die maar één of twee airbnb's in die wijk hebben, wat de gemiddelde prijs kan beïnvloeden. Het kan helpen om een minimum aantal airbnb's per wijk te eisen, dit uit te leggen in de tekst. 
- In hetzelfde kader kan het sowieso handig zijn om een analyse te doen van de spreiding of counts per wijk, zodat je een beter beeld krijgt van de data voordat je gaat visualiseren.

---

In [ ]:
ax = sns.barplot(x="neighbourhood", y="price_numeric", data=airbnb)
plt.xticks(rotation=90)
plt.title("Gemiddelde Airbnb-prijs per wijk")
plt.show()

Het valt op dat er best een grote spreiding in gemiddelde prijs zit tussen verschillende wijken. Zo kost een airbnb in de goedkoopste wijk gemiddeld rond de 50 euro per nacht, en in de duurste wijk ongeveer 775 euro per nacht.

Conclusie: De wijk waarin een airbnb staat heeft zeker invloed op de vraagprijs.

### Visualisatie 2

#### **[FEEDBACK]**

Deze grafiek wordt ook al netjes, maar er mag iets meer uitgelegd worden over de keuzes m.b.t. de visualisatie.

Met name zou ik graag zien waarom je voor een log-log plot hebt gekozen, aangezien dit niet een ideale keuze **kan** zijn voor een dataset die bijvoorbeeld een hoge lineaire correlatie heeft, dit valt veel minder goed te zien op de log-log plot.

Ook is het lastige om de correlatie precies te bepalen door de dichtheid van de punten, maar dit is iets wat lastig te overkomen is.

---

Met deze scatterplot probeer ik aan te tonen of er een verband is tussen het aantal reviews en de prijs van een airbnb:

In [ ]:
sns.scatterplot(x="number_of_reviews", y="price_numeric", data=airbnb)
#plt.xscale("log")   # optioneel, als er hele grote uitschieters zijn
#plt.yscale("log")   # idem
plt.xlabel("Aantal reviews")
plt.ylabel("Prijs")
plt.title("Relatie tussen prijs en aantal reviews")
plt.show()

Het valt op dat er verticale lijnen met datapunten in de plot verschijnen. Dit komt doordat ik een logaritmische schaal gebruik op de x-as.

Conclusie: Er is geen duidelijk verband te zien tussen het aantal reviews en de prijs van een airbnb. Het lijkt erop dat de prijs iets zakt naarmate het aantal reviews stijgt, maar ik vind dat deze trend niet duidelijk genoeg is om er conclusies uit te trekken.